In [1]:
import torch
import numpy as np

## 1.view & reshape

view和reshape是用于改变维度的操作，基本用法是保证：
a.pred(a.size())=a.pred(a'.size())

In [3]:
a = torch.rand(4,1,28,28)

In [6]:
a.view(4,1*28*28).shape

torch.Size([4, 784])

In [8]:
a.reshape(4*1*28,28).shape

torch.Size([112, 28])

## 2.squeeze & unsqueeze

squeeze和unsqueeze是挤压与舒展操作，参数范围在[-a.dim()-1,a.dim()+1)]之间，是正数则在之前插入一个维度，是负数则在之后插入一个维度

In [9]:
a.shape

torch.Size([4, 1, 28, 28])

In [10]:
a.unsqueeze(0).shape  # 在下标0之前插入

torch.Size([1, 4, 1, 28, 28])

In [11]:
a.unsqueeze(-1).shape  # 在最后一个元素之后插入

torch.Size([4, 1, 28, 28, 1])

In [12]:
a.unsqueeze(4).shape

torch.Size([4, 1, 28, 28, 1])

In [13]:
a.unsqueeze(-5).shape

torch.Size([1, 4, 1, 28, 28])

In [14]:
a.unsqueeze(5).shape  # 错误示范

IndexError: Dimension out of range (expected to be in range of [-5, 4], but got 5)

注意实际操作：

In [15]:
a = torch.tensor([1.2, 2.3])
a

tensor([1.2000, 2.3000])

In [20]:
b=a.unsqueeze(-1)
b

tensor([[1.2000],
        [2.3000]])

In [21]:
b.size()

torch.Size([2, 1])

可以看到size由原来的[2]变为[2,1]，此时注意实际张量的变化

In [22]:
c=a.unsqueeze(0)
c

tensor([[1.2000, 2.3000]])

In [23]:
b=torch.rand(32)

In [24]:
f=torch.rand(4,32,14,14)

In [25]:
b = b.unsqueeze(1).unsqueeze(2).unsqueeze(0)
b.shape

torch.Size([1, 32, 1, 1])

## squeeze操作

挤压掉维度为1的维度

In [27]:
torch.rand(1,3,1) # 看一下这个形式如何显示

tensor([[[0.2245],
         [0.0236],
         [0.0166]]])

In [28]:
b.shape

torch.Size([1, 32, 1, 1])

In [29]:
b.squeeze().shape

torch.Size([32])

In [30]:
b.squeeze(0).shape

torch.Size([32, 1, 1])

In [31]:
b.squeeze(-1).shape

torch.Size([1, 32, 1])

## 3.Expand/repeat

维度扩展和重复

### Expand

In [32]:
a = torch.rand(4,32,14,14)

In [33]:
b.shape

torch.Size([1, 32, 1, 1])

In [34]:
b.expand(4,32,14,14).shape

torch.Size([4, 32, 14, 14])

注意下面的演示

In [35]:
a = torch.rand(1,2,1)
a    

tensor([[[0.5072],
         [0.4899]]])

In [36]:
a.expand(2,2,2)

tensor([[[0.5072, 0.5072],
         [0.4899, 0.4899]],

        [[0.5072, 0.5072],
         [0.4899, 0.4899]]])

In [40]:
a.repeat(2,2,2)

tensor([[[0.5072, 0.5072],
         [0.4899, 0.4899],
         [0.5072, 0.5072],
         [0.4899, 0.4899]],

        [[0.5072, 0.5072],
         [0.4899, 0.4899],
         [0.5072, 0.5072],
         [0.4899, 0.4899]]])

In [41]:
a.repeat(2,2,2).shape

torch.Size([2, 4, 2])

注意观察expand和repeat的区别，repeat是指拷贝次数

In [42]:
b.shape

torch.Size([1, 32, 1, 1])

如何用repeat将之变成[4,32,14,14]呢

In [43]:
b.repeat(4,1,14,14).shape

torch.Size([4, 32, 14, 14])

## 4.转置操作.t

In [ ]:
转置只适用于2维

In [47]:
c = torch.rand(2,2)
c

tensor([[0.9741, 0.6815],
        [0.1810, 0.7839]])

In [48]:
c.t()

tensor([[0.9741, 0.1810],
        [0.6815, 0.7839]])

## Transpose

transpose交换两个维度

In [51]:
a = torch.rand(4,3,32,32)

 难点理解

错误使用

In [52]:
a1=a.transpose(1,3).view(4,3*32*32).view(4,3,32,32)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

如果在view之前用了transpose, permute等，需要用contiguous()来返回一个contiguous copy.
一种可能的解释是：
有些tensor并不是占用一整块内存，而是由不同的数据块组成，而tensor的view()操作依赖于内存是整块的，这时只需要执行contiguous()这个函数，把tensor变成在内存中连续分布的形式。

对比

In [55]:
a1 = a.transpose(1,3).contiguous().view(4,3*32*32).view(4,3,32,32)

In [56]:
a2 = a.transpose(1,3).contiguous().view(4,3*32*32).view(4,32,32,3).transpose(1,3)

说明：数据的维度顺序必须和存储顺序一致

验证：

In [57]:
a1.shape,a2.shape

(torch.Size([4, 3, 32, 32]), torch.Size([4, 3, 32, 32]))

In [58]:
torch.all(torch.eq(a, a1))

tensor(False)

In [59]:
torch.all(torch.eq(a, a2))

tensor(True)

说明只有a2是和a全等

In [61]:
a3 = a.transpose(1,3).reshape(4,3*32*32).reshape(4,32,32,3).transpose(1,3)

In [62]:
torch.all(torch.eq(a,a3))

tensor(True)

说明：reshape()相当于contiguous().view() 

## permute

In [65]:
b=torch.rand(4,3,28,32)

In [66]:
b.transpose(1,3).transpose(1,2).shape

torch.Size([4, 28, 32, 3])

使用permute非常简单

In [67]:
b.permute(0,2,3,1).shape

torch.Size([4, 28, 32, 3])